So this is current work. parks (below) is the first 1000 rows of the dataset we have.

We have a couple of issues atm but basically what I want to do is be able to say that given some date in the future, what is the chance that the nearby meters are open based on past data. I want to have a table where the rows are dates/times and the columns are meters and that we can call this table for our model.

One issue is some of the Parking Start Datesand End Dates are the same. I've contacted the city about that but for now im assuming we can proxy it by writing some code to use their parking meter fees ($1 per hour to proxy the time spent)


* Segment by DoW, Hour, Month (onehot) predict by geohash, (predict number of meters filled - where there are nearby areas of high parking)

In [1]:
from datascience import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
#THIS IS IMPORTANT: PM IS ALL THE METERS IN COLUMBUS.
pm = pd.read_csv('Parking_Meters.csv')
pm.head(5)

,X,Y,OBJECTID,METER_ID,LOCATION,SIDE_OF_STREET,BLOCKFACE,METER_STATUS,TOW_AWAY_HOURS,METER_TIME,HANDICAP,HOURS_OPERATION,IN_SERVICE,VALET_HOURS,RATE,FOOD_SERVICE_HOURS,TAXI_ZONE_HOURS,CHARGING_STATION,CHARGING_STATION_STATUS
0,-83.010278,39.960264,6111,C401,STARLING ST S,West,CAPITAL ST - RUSH AL,Operational,3AM - 7AM - 4PM - 6PM,720.0,0.0,6 AM - 6 PM,12.0,NaN,$4.80,NaN,NaN,No,NaN
1,-83.010269,39.960203,6112,C403,STARLING ST S,West,CAPITAL ST - RUSH AL,Operational,3AM - 7AM - 4PM - 6PM,720.0,0.0,6 AM - 6 PM,12.0,NaN,$4.80,NaN,NaN,No,NaN
2,-83.008661,39.957754,5495,C867,TOWN ST W,South,BELLE ST - STARLING ST,Meter & Post Removed,3AM - 7AM - 4PM - 6PM,720.0,0.0,6 AM - 6 PM,12.0,NaN,$4.80,NaN,NaN,No,NaN
3,-83.008583,39.957763,5494,C869,TOWN ST W,South,BELLE ST - STARLING ST,Meter & Post Removed,3AM - 7AM - 4PM - 6PM,720.0,0.0,6 AM - 6 PM,12.0,NaN,$4.80,NaN,NaN,No,NaN
4,-83.008515,39.957769,5493,C871,TOWN ST W,South,BELLE ST - STARLING ST,Meter & Post Removed,3AM - 7AM - 4PM - 6PM,720.0,0.0,6 AM - 6 PM,12.0,NaN,$4.80,NaN,NaN,No,NaN


In [4]:
#SEARCHING FOR PARKING DATASET
searchingforparking = pd.read_csv('urbaninfrastructure_searchingforparking_csv_searchingforparking.csv')
searchingforparking.head(5)

,ParkingGeohash,AvgLatitude,AvgLongitude,City,County,State,Country,ISO_3166_2,AvgTimeToPark,AvgTimeToParkRatio,...,CirclingDistribution,HourlyDistribution,PercentCar,PercentMPV,PercentLDT,PercentMDT,PercentHDT,PercentOther,UpdateDate,Version
0,dphghs8,39.92722,-83.12149,Columbus,Franklin,Ohio,United States of America,US-OH,6.0181,1.1101,...,"{""dphghs3"":0.1006, ""dphghkx"":0.0562, ""dphghs2""...","{""00"": null, ""01"": null, ""02"": null, ""03"": 0.2...",0.0,0.000,0.0,0.000,0.5000,0.5000,2018-05-02,1.1
1,dphgvtg,40.06605,-83.07390,Columbus,Franklin,Ohio,United States of America,US-OH,3.4738,1.0164,...,"{""dphgvtf"":0.0984, ""dphgvtd"":0.1785, ""dphgvw4""...","{""00"": null, ""01"": null, ""02"": null, ""03"": 0.1...",0.0,0.000,0.0,0.000,0.6429,0.3571,2018-05-02,1.1
2,dphgupj,40.07332,-83.13679,Columbus,Franklin,Ohio,United States of America,US-OH,4.0167,1.1655,...,"{""dphgupn"":0.1154, ""dphgupj"":0.3818, ""dphgunu""...","{""00"": null, ""01"": null, ""02"": null, ""03"": nul...",0.0,0.000,0.0,0.000,0.9231,0.0769,2018-05-02,1.1
3,dpj56ve,39.97750,-82.84371,Columbus,Franklin,Ohio,United States of America,US-OH,4.8716,3.4115,...,"{""dpj56ve"":0.172, ""dpj56vg"":0.2548, ""dpj56vf"":...","{""00"": null, ""01"": null, ""02"": null, ""03"": nul...",0.0,0.125,0.0,0.125,0.6250,0.1250,2018-05-02,1.1
4,dpj5391,39.95306,-82.90108,Columbus,Franklin,Ohio,United States of America,US-OH,4.6896,1.0568,...,"{""dpj5393"":0.0191, ""dpj5390"":0.0621, ""dpj5391""...","{""00"": null, ""01"": null, ""02"": null, ""03"": nul...",0.0,0.000,0.0,0.000,0.8750,0.1250,2018-05-02,1.1


In [6]:
#FULL DATASET
full_data = pd.read_csv('parking_meter_data_2015_-_2017.csv')
#full_data = full_data.drop([1523047, 3195633,4863068])
full_data.head(5)

C:\Users\Derek\Miniconda3\envs\ds100\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Pole,ParkingStartDate,ParkingEndDate,TransactionType,TotalCredit
0,SI83,2015-01-01 00:09:24,2015-01-01 00:09:24,Cash,0.5
1,UE255,2015-01-01 00:21:22,2015-01-01 00:21:22,Cash,0.75
2,EG6,2015-01-01 00:26:42,2015-01-01 00:26:42,Cash,0.05
3,EG6,2015-01-01 00:27:02,2015-01-01 00:27:02,Cash,0.05
4,UE242,2015-01-01 00:27:16,2015-01-01 00:27:16,Cash,0.25


In [ ]:
#ADDING DIFFERENCE IN DATES
full_data['DifferenceAsDateTime'] = pd.to_datetime(full_data['ParkingEndDate'], errors='coerce') - pd.to_datetime(full_data['ParkingStartDate'], errors='coerce')


In [ ]:
full_data.groupby("Pole").count().sort_values(by=["TotalCredit"])

In [ ]:
a,b,c = [],[],[]
a2,b2,c2 = [],[],[]
for each in np.arange(len(full_data)):
    if each not in [1523047, 3195633,4863068]:
        a.append(full_data["ParkingStartDate"][each][0:10])
        b.append(full_data["ParkingStartDate"][each][11:13])
        c.append(full_data["ParkingStartDate"][each][14:16])
        a2.append(full_data["ParkingEndDate"][each][0:10])
        b2.append(full_data["ParkingEndDate"][each][11:13])
        c2.append(full_data["ParkingEndDate"][each][14:16])
    else: 
        a.append(full_data["ParkingStartDate"][0][0:10])
        b.append(full_data["ParkingStartDate"][0][11:13])
        c.append(full_data["ParkingStartDate"][0][14:16])
        a2.append(full_data["ParkingEndDate"][0][0:10])
        b2.append(full_data["ParkingEndDate"][0][11:13])
        c2.append(full_data["ParkingEndDate"][0][14:16])
full_data["StartDate"] = a
full_data["StartHour"] =  b
full_data["StartMinute"] =  c
full_data[ "EndDate"] = a2 
full_data["EndHour"] =  b2 
full_data["EndMinute"] =  c2
          
full_data.head(5)

In [ ]:
# TIMES IN OPERATION
times_op = []
for eachH in np.arange(6,22):
    for eachM in np.arange(0,60):
        times_op.append([eachH, eachM])
len(times_op)

In [ ]:
#THIS CODE NEEDS TO BE ADJUSTED BUT BASICALLY IT GOES THROUGH EVERY TIME IN OPERATION, 
# FOR EVERY DATE FOR A SPECIFIC METER AND TELLS YOU THE PROPORTION OF TIME THAT METER WAS OPEN VS TAKEN..

def when_is_the_station_open(pole):
    
    temp = full_data[full_data["Pole"] == pole]
  
    lst_of_lsts = []
    for helper in np.unique(full_data["StartDate"]):
        #print(helper)
        temp1 = temp[temp["StartDate"] == helper][['StartHour','StartMinute','EndHour','EndMinute']]
        lst = []
        timechecktest = []
        ans = []
        for time_tobe_checked in times_op:
            for each in np.arange(len(temp1)):

                # IF START HOUR IS THE SAME AND IF END HOUR IS THE SAME
                        # THEN IF START MIN IS LESS OR EQUAL  (ELSE FALSE)
                            # THEN IF END MIN IS MORE OR EQUAL - TRUE (ELSE FALSE)
                     
                     #b1temp1.iloc[0]["StartHour"]

                if (int(temp1.iloc[each]["StartHour"]) == int(time_tobe_checked[0])) & (int(temp1.iloc[each]["EndHour"]) == int(time_tobe_checked[0])):
                    if (int(time_tobe_checked[1]) >= int(temp1.iloc[each]["StartMinute"])):
                        if (int(time_tobe_checked[1]) <= int(temp1.iloc[each]["EndMinute"])):
                            lst.append("Taken")
                            #print(time_tobe_checked)
                            timechecktest.append(time_tobe_checked)
                        else:
                            lst.append("Open")
                    else:
                        lst.append("Open")

                 # IF START HOUR IS SAME AND END HOUR IS DIFFERENT
                    # THEN IF START MIN IS LESS OR EQUAL - TRUE
                    # ELSE FALSE

                elif (int(temp1.iloc[each]["StartHour"]) == int(time_tobe_checked[0])) & (int(temp1.iloc[each]["EndHour"]) != int(time_tobe_checked[0])):
                    if (int(time_tobe_checked[1]) >= int(temp1.iloc[each]["StartMinute"])):
                        lst.append("Taken")
                        #print(time_tobe_checked)
                        timechecktest.append(time_tobe_checked)
                    else:
                        lst.append("Open")

                # IF START HOUR IS DIFFERENT AND END HOUR IS SAME 
                    # THEN IF END MIN IS MORE OR EQUAL - TRUE
                    # ELSE FALSE


                elif (int(temp1.iloc[each]["StartHour"]) != int(time_tobe_checked[0])) & (int(temp1.iloc[each]["EndHour"]) == int(time_tobe_checked[0])):
                    if (int(time_tobe_checked[1]) <= int(temp1.iloc[each]["EndMinute"])):
                        lst.append("Taken")
                        #print(time_tobe_checked)
                        timechecktest.append(time_tobe_checked)
                    else:
                        lst.append("Open")

            # IF START HOUR IS DIFFERENT AND END HOUR IS DIFFERENT - AUTO FALSE
                else:
                    lst.append("Open")
        for each_ in times_op:
            if each_ in timechecktest:
                #print("Taken - ",each_)
                ans.append([1,each_])
            else:
                #print("Open - ",each_)
                ans.append([0,each_])
        lst_of_lsts.append(ans)
    return lst_of_lsts
